In [42]:
import nest_asyncio
nest_asyncio.apply()

In [43]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import functools
import collections
import os

In [44]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential

In [45]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [46]:
gpu_devices = tf.config.list_physical_devices('GPU')
if not gpu_devices:
  print('Cannot detect physical GPU device in TF')
# tf.config.set_logical_device_configuration(
#     gpu_devices[0], 
#     [tf.config.LogicalDeviceConfiguration(memory_limit=5120),
#      tf.config.LogicalDeviceConfiguration(memory_limit=5120)])
# tf.config.list_logical_devices()


Cannot detect physical GPU device in TF


In [47]:
#setting uop tensorboard 
 
# import os 
# import shutil

# logdir = "C:\Users\sidha\Stevens\AAI800 - Special Projects in AI\Project Execution\tensorboard"
# if os.path.exists(logdir):
#     shutil.rmtree(logdir)

# summary_writer = tf.summary.create_file_writer(logdir)
# state = iter

In [48]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()


b'Hello, World!'

In [49]:
default_client_dataset_size = 500
MAX_CLIENT_DATASET_SIZE = int(input("Enter Max Client Datset Size: ") or default_client_dataset_size)

default_client_epochs_per_round = 1
CLIENT_EPOCHS_PER_ROUND = int(input("Enter Client Epochs pe round: ") or default_client_epochs_per_round)

default_client_batch_size = 20
CLIENT_BATCH_SIZE = int(input("Enter Client Batch Size: ") or default_client_batch_size)

default_client_test_batch_size = 100
TEST_BATCH_SIZE = int(input("Enter Test Batch Size: ") or default_client_test_batch_size)

NUM_CLIENTS = 2

In [50]:
cifar_train, cifar_test = tff.simulation.datasets.cifar100.load_data(cache_dir=None)

In [51]:
cifar_train.element_type_structure

OrderedDict([('coarse_label', TensorSpec(shape=(), dtype=tf.int64, name=None)),
             ('image',
              TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None)),
             ('label', TensorSpec(shape=(), dtype=tf.int64, name=None))])

In [52]:
example_dataset = cifar_train.create_tf_dataset_for_client(cifar_train.client_ids[0])
example_element= next(iter(example_dataset))
example_element['label'].numpy()

75

In [53]:
example_element['image']

<tf.Tensor: shape=(32, 32, 3), dtype=uint8, numpy=
array([[[ 89,  71,  73],
        [120,  98, 106],
        [137, 113, 116],
        ...,
        [133, 105, 102],
        [131, 104,  98],
        [142, 121, 107]],

       [[ 94,  76,  75],
        [123, 101, 105],
        [169, 144, 145],
        ...,
        [152, 123, 120],
        [133, 107, 101],
        [168, 146, 137]],

       [[133, 113, 110],
        [151, 128, 130],
        [147, 121, 120],
        ...,
        [167, 139, 133],
        [164, 138, 130],
        [197, 174, 168]],

       ...,

       [[ 87,  69,  74],
        [102,  82,  85],
        [ 97,  73,  75],
        ...,
        [175, 158, 152],
        [143, 124, 121],
        [158, 134, 130]],

       [[ 87,  68,  69],
        [100,  79,  79],
        [110,  86,  84],
        ...,
        [148, 131, 123],
        [160, 143, 143],
        [172, 155, 157]],

       [[ 93,  74,  71],
        [157, 136, 131],
        [170, 145, 139],
        ...,
        [164, 147, 136]

In [54]:
# def reshape_cifar_element(element):
#   return (tf.expand_dims(element['image'], axis=-1), element['coarse_label'])
def reshape_cifar_element(element):
 """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
 x=tf.reshape(element['image'], [-1, 3072])
 x=tf.cast(x,dtype=tf.float32)
 x /= 255
 y=tf.reshape(element['coarse_label'], [-1, 1])
 return collections.OrderedDict( 
    x=(tf.cast(tf.reshape(element['image'], [-1, 3072]),dtype=tf.float32))/255,
    y=tf.reshape(element['label'], [-1, 1]))
        


In [55]:
def preprocess_dataset(dataset):
  return (dataset
          .shuffle(buffer_size=MAX_CLIENT_DATASET_SIZE)
          .repeat(CLIENT_EPOCHS_PER_ROUND)
          .batch(CLIENT_BATCH_SIZE, drop_remainder=False)
          .map(reshape_cifar_element)
          .prefetch(10))

In [56]:
preprocessed_example_dataset = preprocess_dataset(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch


OrderedDict([('x',
              array([[0.42745098, 0.49803922, 0.7372549 , ..., 0.5529412 , 0.6901961 ,
                      0.9411765 ],
                     [0.04705882, 0.1254902 , 0.4627451 , ..., 0.2       , 0.27058825,
                      0.39607844],
                     [0.99607843, 0.99607843, 0.99607843, ..., 0.9843137 , 0.9843137 ,
                      0.9843137 ],
                     ...,
                     [0.49411765, 0.54509807, 0.5529412 , ..., 0.59607846, 0.5058824 ,
                      0.4862745 ],
                     [0.3137255 , 0.54509807, 0.88235295, ..., 0.28235295, 0.39215687,
                      0.18431373],
                     [0.2509804 , 0.2509804 , 0.2509804 , ..., 0.3254902 , 0.3254902 ,
                      0.3254902 ]], dtype=float32)),
             ('y',
              array([[23],
                     [23],
                     [29],
                     [71],
                     [93],
                     [51],
                     [93

In [57]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess_dataset(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [58]:
sample_clients = cifar_train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(cifar_train, sample_clients)

print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

Number of client datasets: 2
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 3072)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int64)])>


In [59]:
# cifar_train_federated = cifar_train.preprocess(preprocess_train_dataset)
# cifar_train_federated

In [60]:
# def create_original_fedavg_cnn_model(cache_dir=None):
#   data_format = 'channels_last'

#   max_pool = functools.partial(
#       tf.keras.layers.MaxPooling2D,
#       pool_size=(2, 2),
#       padding='same',
#       data_format=data_format)
#   conv2d = functools.partial(
#       tf.keras.layers.Conv2D,
#       kernel_size=5,
#       padding='same',
#       data_format=data_format,
#       activation=tf.nn.relu)
#   model = tf.keras.applications.resnet.ResNet101(include_top=False,
#                                                  weights='imagenet',
#                                                  input_tensor=None,
#                                                  input_shape=(32,32,3),
#                                                  pooling=max,
#                                                  classes=20,
#                                                  )
  
#   return model

In [61]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

BatchNormalization()
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(100))


model.add(Activation('softmax'))


In [62]:
# Model=create_original_fedavg_cnn_model()
Model=model

In [63]:
Model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 12, 12, 64)       

In [64]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  return tff.learning.from_keras_model(
      Model,
      input_spec= preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]) 

In [65]:
cifar100Variables = collections.namedtuple('cifar100Variables','weights bias num_examples loss_sum accuracy_sum')

In [66]:
def create_cifar100_variables():
    return cifar100Variables(
      weights=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(3072,100)),
          name='weights',
          trainable=True),
      bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(100)),
          name='bias',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False)
    )

In [67]:
def batch_model(variables,x):
    return tf.nn.softmax(tf.matmul(x, variables.weights) + variables.bias)
    # tff.learning.from_keras_model(
    #   keras_model=Model(),
    #   input_spec=input_spec,
    #   loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    #   metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
      
def cifar100_forward_pass(variables, batch):
  y = batch_model(variables, batch['x'])
  predictions = tf.cast(tf.argmax(y, 1), tf.int64)

  flat_labels = tf.reshape(batch['y'], [-1])
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 100) * tf.math.log(y), axis=[1]))
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))

  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  return loss, predictions

In [68]:
def get_local_unfinalized_metrics(variables):
  return collections.OrderedDict(
      num_examples=[variables.num_examples],
      loss=[variables.loss_sum, variables.num_examples],
      accuracy=[variables.accuracy_sum, variables.num_examples])

In [69]:
def get_metric_finalizers():
  return collections.OrderedDict(
      num_examples=tf.function(func=lambda x: x[0]),
      loss=tf.function(func=lambda x: x[0] / x[1]),
      accuracy=tf.function(func=lambda x: x[0] / x[1]))

In [70]:
def get_local_mnist_metrics(variables):
  return collections.OrderedDict(
      num_examples=variables.num_examples,
      loss=variables.loss_sum / variables.num_examples,
      accuracy=variables.accuracy_sum / variables.num_examples)

In [71]:
@tff.federated_computation
def aggregate_mnist_metrics_across_clients(metrics):
  return collections.OrderedDict(
      num_examples=tff.federated_sum(metrics.num_examples),
      loss=tff.federated_mean(metrics.loss, metrics.num_examples),
      accuracy=tff.federated_mean(metrics.accuracy, metrics.num_examples))

In [72]:
from typing import Callable, List, OrderedDict

class cifar100Model(tff.learning.Model):

  def __init__(self):
    self._variables = create_cifar100_variables()

  @property
  def trainable_variables(self):
    return [self._variables.weights, self._variables.bias]

  @property
  def non_trainable_variables(self):
    return []

  @property
  def local_variables(self):
    return [
        self._variables.num_examples, self._variables.loss_sum,
        self._variables.accuracy_sum
    ]

  @property
  def input_spec(self):
    return collections.OrderedDict(
        x=tf.TensorSpec([None, 3072], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int64))

  @tf.function
  def predict_on_batch(self, x, training=True):
    del training
    return batch_model(self._variables, x)
    
  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = cifar100_forward_pass(self._variables, batch)
    num_examples = tf.shape(batch['x'])[0]
    return tff.learning.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_examples)
        
  @tf.function
  def report_local_outputs(self):
    return get_local_mnist_metrics(self._variables)

  @property
  def federated_output_computation(self):
    return aggregate_mnist_metrics_across_clients


  @tf.function
  def report_local_unfinalized_metrics(
      self) -> OrderedDict[str, List[tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to unfinalized values."""
    return get_local_unfinalized_metrics(self._variables)

  def metric_finalizers(
      self) -> OrderedDict[str, Callable[[List[tf.Tensor]], tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to finalizers."""
    return get_metric_finalizers()

  @tf.function
  def reset_metrics(self):
    """Resets metrics variables to initial value."""
    for var in self.local_variables:
      var.assign(tf.zeros_like(var))

In [85]:
# training the model 
federated_averaging = tff.learning.(
    model_fn=cifar100Model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02)
    # server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=5.0)
    # use_experimental_simulation_loop=True
    )

In [86]:
state = federated_averaging.initialize()

In [87]:

state, metrics = federated_averaging.next(state, federated_train_data)
    # result = iterative_process.next(state, federated_train_data)
    # state = result.state
    # metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('num_examples', 200.0), ('loss', 3.9105146), ('accuracy', 0.075)]))])


In [88]:

for round_num in range(2, 11):
    state, metrics = federated_averaging.next(state, federated_train_data)
        # result = iterative_process.next(state, federated_train_data)
        # state = result.state
        # metrics = result.metrics
    print('round {:2d}, metrics={}'.format(round_num,metrics))

round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('num_examples', 200.0), ('loss', 3.5689206), ('accuracy', 0.095)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('num_examples', 200.0), ('loss', 3.5829897), ('accuracy', 0.105)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('num_examples', 200.0), ('loss', 3.2940521), ('accuracy', 0.11)]))])
round  5, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('num_examples', 200.0), ('loss', 3.0494468), ('accuracy', 0.15)]))])
round  6, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict(

%##############################################


In [89]:
evaluation = tff.learning.build_federated_evaluation(cifar100Model)
print(evaluation.type_signature.formatted_representation())

(<
  server_model_weights=<
    trainable=<
      float32[3072,100],
      float32[100]
    >,
    non_trainable=<>
  >@SERVER,
  federated_dataset={<
    x=float32[?,3072],
    y=int64[?,1]
  >*}@CLIENTS
> -> <
  num_examples=float32@SERVER,
  loss=float32@SERVER,
  accuracy=float32@SERVER
>)


In [91]:
model_weights = federated_averaging.get_model_weights(state)
train_metrics = evaluation(model_weights, federated_train_data)

AttributeError: 'IterativeProcess' object has no attribute 'get_model_weights'

In [97]:
model_weights = tff.learning.ModelWeights(trainable=state.model,non_trainable=state.model)
# train_metrics = evaluation(model_weights, federated_train_data)
type(model_weights)

tensorflow_federated.python.learning.model_utils.ModelWeights

In [101]:
model_weights

ModelWeights(trainable=ModelWeights(trainable=[array([[ 0.00039498, -0.00067911, -0.00067911, ..., -0.00067911,
        -0.00067911,  0.00221748],
       [ 0.00033089, -0.00074313, -0.00074313, ..., -0.00074313,
        -0.00074313,  0.00134677],
       [-0.00044445, -0.00071834, -0.00071834, ..., -0.00071834,
        -0.00071834,  0.00116415],
       ...,
       [ 0.0007081 , -0.00077924, -0.00077924, ..., -0.00077924,
        -0.00077924,  0.00153356],
       [ 0.00077423, -0.00078516, -0.00078516, ..., -0.00078516,
        -0.00078516,  0.00098196],
       [ 0.00092239, -0.00070942, -0.00070942, ..., -0.00070942,
        -0.00070942,  0.00107656]], dtype=float32), array([ 0.00099185, -0.00186921, -0.00186921, -0.00186921, -0.00186921,
       -0.00186921, -0.00186921, -0.00186921, -0.00186921, -0.00034812,
        0.0024676 , -0.00186921, -0.00186921, -0.00186921, -0.00186921,
        0.00122479,  0.00070398, -0.00186921, -0.00186921, -0.00186921,
       -0.00186921,  0.00120148, -0.

In [103]:
# model_weights_2 = tf.TensorSpec(model_weights)
train_metrics = evaluation(model_weights.trainable, federated_train_data)